In [1]:
import pandas as pd
import glob
import os
import numpy as np
import pickle
import statistics
import community
import matplotlib as mlp
# mlp.use("Qt5Agg")
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx

import analysis as an
import pandas as pd


import numpy as np
import seaborn as sns

from scipy import stats 
import scipy
import pprint as pp
import bct as bct

In [2]:
basepath='/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/'


In [ ]:
#making a dataframe
_dfs=[]
for group, data in subedge_dict.items():
    print(group)
    for sub, dat in data['graphs'].items():
        print(sub)
        tmp = pd.DataFrame.from_dict(dict(dat.nodes(data =True)), orient='index')
        tmp['subject'] = sub
        tmp['group'] = group
        tmp['IC'] = tmp.index
#         tmp['IC']='IC_%i'%tmp['IC']
        tmp['IC'] = 'IC_' + tmp['IC'].astype(str)
        _dfs.append(tmp)
total = pd.concat(_dfs)
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'demo*')
demo_dict=an.onetoughjar(latest_file)
demo_df = pd.DataFrame.from_dict(demo_dict['NR'], orient='index')
demo_df['subject'] = demo_df.index
demo_df.reset_index(inplace = True)
demo_df["subject"]=pd.to_numeric(demo_df["subject"])
total["subject"]=pd.to_numeric(total["subject"])
complete_df=pd.merge(demo_df, total, on="subject")
#complete_df.to_csv(os.path.join(basepath,'tmp','complete_data.csv'), sep=',')
#an.adillyofapickle('/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets',complete_df,'11_complete_df')


In [ ]:
def zscore(col):
    col_z = (col - col.mean())/col.std(ddof=0)
    return(col_z)

In [3]:
p='/Users/gracer/Google Drive/HCP/HCP_graph/1200/'
atlas=pd.read_csv(os.path.join(p,'brains','atlas.csv'), sep=',')


In [3]:
labels = pd.read_csv('/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/mod_labels.csv', sep=',')

In [4]:
# this is a dataframe
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'6_*')
submod_dict=an.onetoughjar(latest_file)

/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/6_submod_dict_09-15-2020_03-49-48


In [5]:
# this is the graph
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'5_*')
summary_dict=an.onetoughjar(latest_file)

/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/5_summary_dict_09-15-2020_03-49-44


In [6]:
# this is the graph
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'7_*')
subgraph_dict=an.onetoughjar(latest_file)

/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/7_subgraph_dict_09-17-2020_08-55-55


In [7]:
# this is the graph
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'10_*')
subedge_dict=an.onetoughjar(latest_file)

/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/10_subedge_dict_09-17-2020_09-43-31


In [ ]:
# this is the graph
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'10_*')
subedge_dict=an.onetoughjar(latest_file)

In [ ]:
subedge_dict['no'].keys()

In [ ]:
test = dict(subedge_dict['no']['graphs']['138837'].nodes(data=True))

In [ ]:
pd.DataFrame.from_dict(test, orient='index')

In [ ]:
_dfs=[]
for group, data in subedge_dict.items():
    print(group)
    for sub, dat in data['graphs'].items():
        print(sub)
        tmp = pd.DataFrame.from_dict(test, orient='index')
        tmp['subject'] = sub
        tmp['group'] = group
        tmp['IC'] = tmp.index
        tmp['IC']=tmp['IC']+1
        _dfs.append(tmp)
total = pd.concat(_dfs)

In [ ]:
total

In [ ]:
demo_dict['NR'].keys()

In [ ]:
demo_df = pd.DataFrame.from_dict(demo_dict['NR'], orient='index')

In [ ]:
demo_df['subject'] = demo_df.index
demo_df.reset_index(inplace = True)

In [ ]:
demo_df

In [ ]:
demo_df["subject"]=pd.to_numeric(demo_df["subject"])
total["subject"]=pd.to_numeric(total["subject"])

In [ ]:
pd.merge(demo_df, total, on="subject")

In [ ]:
x

In [ ]:
part = pd.DataFrame.from_dict(summary_dict['no']['modules']['partition'], orient='index', columns = ['module'])


Making an edge data frame

In [ ]:
edges = {}

for group, stuff in summary_dict.items():
    print(group)
    _df = nx.to_pandas_edgelist(stuff['comm_graph'])
    _df.loc[(_df['source'] == _df['target']), 'weight'] = 0

    _df['group']=group
    edges[group]=_df

edge_df=pd.concat(list(edges.values()))

In [ ]:
edge_df['z_weight']=an.zscore(edge_df['weight'])

# Set normalized edge as an get_edge_attributes
for k,v in update_dict.items():
    test=edge_df[edge_df['group']==k]
    keyz = list(zip(test['source'],test['target']))
    values=test['z_weight']
    up_dict={}
    for i in range(len(keyz)):
        up_dict[keyz[i]]={'z_edge':values[i]}
    nx.set_edge_attributes(v['comm_graph'], up_dict)


In [ ]:
edge_df

Checking the save dict to calculcate differences in edges 

In [8]:
# this is a dataframe
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'3_*')
save_dict=an.onetoughjar(latest_file)

/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/3_save_dict_08-31-2020_09-48-50


In [10]:
for group, values in save_dict['NR'].items():
    print(group)
    values['modules'] = summary_dict[group]['modules']['partition']
    for sub, graph in values['graphs'].items():
        print(sub)
        G = graph
        modules = summary_dict[group]['modules']['partition']
        nx.set_node_attributes(G, modules, 'modules')

no
200513
379657
125222
217429
157437
979984
256540
281135
117324
140319
285446
729557
130821
145127
529549
188145
645551
146331
137027
309636
168139
198451
152427
173536
704238
186444
371843
105620
475855
180735
154734
361234
129634
174437
835657
245333
178950
210112
194847
185846
148840
135629
378857
553344
120717
138332
138534
169545
135225
561444
622236
238033
206222
204420
117122
178849
109830
228434
173940
208125
102109
381543
571144
499566
465852
386250
180432
350330
227432
448347
555651
459453
366042
156536
128127
101107
185341
173637
101915
214726
162228
198855
122620
837560
144125
849971
144428
112112
131419
580347
142828
200917
749361
172534
199352
107321
316633
176037
558960
616645
654754
160123
349244
361941
186040
157942
113619
190031
432332
101309
185038
727553
158843
479762
149842
138231
397154
336841
114318
120414
175742
153429
715041
210011
567759
132017
802844
122317
250932
214423
285345
130013
111716
175338
146533
173738
211417
208226
180937
114823
176441
173435
148

In [ ]:
save_dict['NR']['no'].keys()

In [ ]:
edges = {}

for group, stuff in save_dict['NR'].items():
    print(group)
    for sub, graph in stuff['graphs'].items():
        print(sub)
        _df = nx.to_pandas_edgelist(graph)
        _df.loc[(_df['source'] == _df['target']), 'weight'] = 0

        _df['group']=group
        edges[group]=_df

sub_edge_df=pd.concat(list(edges.values()))

In [ ]:
sub_edge_df = sub_edge_df[sub_edge_df['source'] != sub_edge_df['target']]

In [ ]:
#sub_edge_df.to_csv(os.path.join(basepath,'tmp','sub_edge_data.csv'), sep=',')

In [ ]:
sub_edge_df['z_weight']=an.zscore(sub_edge_df['weight'])

In [ ]:
# Set normalized edge as an get_edge_attributes
for group,v in save_dict['NR'].items():
    for sub, graph in v['graphs'].items():
        test=sub_edge_df[sub_edge_df['group']==group]
        keyz = list(zip(test['source'],test['target']))
        values=test['z_weight']
        up_dict={}
        for i in range(len(keyz)):
            up_dict[keyz[i]]={'z_edge':values[i]}
        nx.set_edge_attributes(graph, up_dict)


Make individual comm graphs

In [ ]:
save_dict['NR']['no']['comm_graph']['200513'].edges(data=True)

In [11]:
# Make community graph
for group,v in save_dict['NR'].items():
    print(group)
    v.update(comm_graph = {})
    for sub, data in v['graphs'].items():
        print(sub)
        comm_graph = community.induced_graph(v['modules'], data)
        v['comm_graph'][sub] = comm_graph


no
200513
379657
125222
217429
157437
979984
256540
281135
117324
140319
285446
729557
130821
145127
529549
188145
645551
146331
137027
309636
168139
198451
152427
173536
704238
186444
371843
105620
475855
180735
154734
361234
129634
174437
835657
245333
178950
210112
194847
185846
148840
135629
378857
553344
120717
138332
138534
169545
135225
561444
622236
238033
206222
204420
117122
178849
109830
228434
173940
208125
102109
381543
571144
499566
465852
386250
180432
350330
227432
448347
555651
459453
366042
156536
128127
101107
185341
173637
101915
214726
162228
198855
122620
837560
144125
849971
144428
112112
131419
580347
142828
200917
749361
172534
199352
107321
316633
176037
558960
616645
654754
160123
349244
361941
186040
157942
113619
190031
432332
101309
185038
727553
158843
479762
149842
138231
397154
336841
114318
120414
175742
153429
715041
210011
567759
132017
802844
122317
250932
214423
285345
130013
111716
175338
146533
173738
211417
208226
180937
114823
176441
173435
148

In [12]:
# Make community graph
for group,v in save_dict['NR'].items():
    print(group)
    v.update(comm_graph = {})
    for sub, data in v['graphs'].items():
        print(sub)
        comm_graph = community.induced_graph(v['modules'], data)
        v['comm_graph'][sub] = comm_graph


no
200513
379657
125222
217429
157437
979984
256540
281135
117324
140319
285446
729557
130821
145127
529549
188145
645551
146331
137027
309636
168139
198451
152427
173536
704238
186444
371843
105620
475855
180735
154734
361234
129634
174437
835657
245333
178950
210112
194847
185846
148840
135629
378857
553344
120717
138332
138534
169545
135225
561444
622236
238033
206222
204420
117122
178849
109830
228434
173940
208125
102109
381543
571144
499566
465852
386250
180432
350330
227432
448347
555651
459453
366042
156536
128127
101107
185341
173637
101915
214726
162228
198855
122620
837560
144125
849971
144428
112112
131419
580347
142828
200917
749361
172534
199352
107321
316633
176037
558960
616645
654754
160123
349244
361941
186040
157942
113619
190031
432332
101309
185038
727553
158843
479762
149842
138231
397154
336841
114318
120414
175742
153429
715041
210011
567759
132017
802844
122317
250932
214423
285345
130013
111716
175338
146533
173738
211417
208226
180937
114823
176441
173435
148

In [13]:
basepath='/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/'
latest_file2=an.find_latest(os.path.join(basepath,'tmp'),'5_*')
summary_dict=an.onetoughjar(latest_file2)
update_dict = {}
for key, value in summary_dict.items():
    print(key)
    update_dict[key] = {**value[0], **value[1]}


/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/5_summary_dict_09-15-2020_03-49-44
no


KeyError: 0

In [19]:
aes_dict={'no':{},
          'ov':{},
          'ob':{}}
for group, stuff in summary_dict.items():
    print(group)
    G=stuff['comm_graph']
    aes_dict[group]=an.aesthetics(G,15000,100, 'sans-serif', 'Bold', 'z_edge', (80,50), 1)


no
8
ov
9
ob
9


In [16]:
summary_dict['no'].keys()

dict_keys(['mean_FC', 'graphs', 'modules', 'corrmat', 'clustering_coeff', 'btn_centrality', 'PC', 'comm_graph'])

In [21]:
# an.module_fig(summary_dict['no']['comm_graph'], 'Average BMI z-scored', basepath, aes_dict['no'])
an.module_fig(summary_dict['ov']['comm_graph'], 'High BMI z-scored', basepath, aes_dict['ov'])
an.module_fig(summary_dict['ob']['comm_graph'], 'Very High BMI z-scored', basepath, aes_dict['ob'])


9
9
[0.44444444 1.33333333 2.22222222 3.11111111 4.         4.88888889
 5.77777778 6.66666667 7.55555556]
9
9
[0.44444444 1.33333333 2.22222222 3.11111111 4.         4.88888889
 5.77777778 6.66666667 7.55555556]


In [ ]:
def cal_edges(dic, basepath, group):
    edges = {}
    for sub, graph in dic.items():
        print(sub)
        _df = nx.to_pandas_edgelist(graph)
        _df.loc[(_df['source'] == _df['target']), 'weight'] = 0
        _df = _df[_df['source'] != _df['target']]
        _df['group']=group
        _df['z_weight']=an.zscore(_df['weight'])
        edges[sub]=_df
    return(edges)
    
    


    

In [ ]:
save_dict['NR']['no']['comm_graph']['157437']

In [ ]:
tmp_dict = {'no':{}, 'ov': {}, 'ob': {}}
for group, dat in save_dict['NR'].items():
    print(group)
    x=cal_edges(dat['comm_graph'], basepath, group)
    tmp_dict[group]=pd.concat(list(x.values()))


In [ ]:
tmp_dict['ob']

In [ ]:
sub_comm_edge_df=pd.concat(list(tmp_dict.values()))


In [ ]:
#sub_comm_edge_df.to_csv(os.path.join(basepath,'tmp','sub_comm_edge_data.csv'), sep=',')

In [ ]:
edges = {}

for group, stuff in save_dict['NR'].items():
    print(group)
    for sub, graph in stuff['graphs'].items():
        print(sub)
        _df = nx.to_pandas_edgelist(graph)
        _df.loc[(_df['source'] == _df['target']), 'weight'] = 0

        _df['group']=group
        edges[group]=_df

sub_edge_df=pd.concat(list(edges.values()))
sub_edge_df = sub_edge_df[sub_edge_df['source'] != sub_edge_df['target']]
# sub_edge_df['z_weight']=an.zscore(sub_edge_df['weight'])
# sub_edge_df.to_csv(os.path.join(basepath,'tmp','sub_edge_data.csv'), sep=',')


In [ ]:
# Set normalized edge as an get_edge_attributes
for group,v in save_dict['NR'].items():
    for sub, graph in v['graphs'].items():
        test=sub_edge_df[sub_edge_df['group']==group]
        keyz = list(zip(test['source'],test['target']))
        values=test['z_weight']
        up_dict={}
        for i in range(len(keyz)):
            up_dict[keyz[i]]={'z_edge':values[i]}
        nx.set_edge_attributes(graph, up_dict)


In [ ]:
#an.adillyofapickle('/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets',save_dict['NR'],'10_subedge_dict')

In [ ]:
def mod_loc(group, module): 
    columns=['Index','area', 'label']
    data = []
    part = pd.DataFrame.from_dict(summary_dict[group]['modules']['partition'], orient='index', columns = ['module'])
    x = part.loc[part['module'] == module].index
    for item in x:
        data.append(labels[['Index','area', 'label']].loc[labels['Index'][item]])    
    df = pd.DataFrame(data, columns=columns)
    return(df)

In [ ]:
mod_loc('no', 0)

# Graphs! Figures!

In [12]:
aes_dict={'no':{},
          'ov':{},
          'ob':{}}
for group, stuff in summary_dict.items():
    print(group)
    G=stuff['comm_graph']
    aes_dict[group]=an.aesthetics(G,15000,100, 'sans-serif', 'Bold', 'z_edge', (80,50), 1)

no
8
ov
9
ob
9


In [ ]:
groups = ['no','ov','ob']
mods = [2,4,0,5,2,6]
IC_=[11,16, 27, 35, 54, 58, 85]
for group in groups:
    for mod in mods:
        for x in IC_:
            if x in list(subgraph_dict[group][mod].nodes()):
                print('for %s in mod %i and node %i'%(group, mod, x))
                print(subgraph_dict[group][mod].nodes(data=True)[x])

In [10]:
def aesthetics(graph, node_size, font_size, font_family, font_weight, edge_att, plot_size, mod):
    aes={'general':{},
         'nodes':{},
         'edges':{}}
    # nodes
    color = np.array(list(graph.nodes))
    color = np.array(color)
    n_color=len(list(set(color)))
    print(n_color)
    aes['nodes']['color'] = color
    aes['nodes']['colormap'] = ['Set3', n_color]
    aes['nodes']['n_color'] = n_color
    aes['nodes']['max'] = float(color.max())
    aes['nodes']['min'] = float(color.min())
    aes['nodes']['font_size'] = int(font_size)
    aes['nodes']['font_family'] = font_family
    aes['nodes']['font_weight'] = font_weight
    aes['nodes']['node_size'] = int(node_size)
    #edges
    aes['edges']['colormap'] = plt.cm.gist_rainbow
    edges,weights = zip(*nx.get_edge_attributes(graph,edge_att).items())
    weights=np.array(weights)
    aes['edges']['width mod'] = mod
    aes['edges']['min'] = weights.min()
    aes['edges']['max'] = weights.max()
    # General
    aes['general']['plot_size']=plot_size #tuple 80,50
    return(aes)

In [ ]:
def module_fig(G, Type, basepath, aes):
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    #nodes, size = zip(*nx.get_node_attributes(G,'clustering').items())

    positions=nx.circular_layout(G)
    plt.figure(figsize= aes['general']['plot_size'])
    ### NODES ####
    print(len(G.nodes()))
    y=nx.draw_networkx_nodes(
        G,positions,
        node_color = aes['nodes']['color'],
        node_size = aes['nodes']['node_size'],
        alpha=1.0,
        cmap= plt.get_cmap('Set3', aes['nodes']['n_color']),
        vmin=aes['nodes']['min'],
        vmax=aes['nodes']['max'])

    #Styling for node labels
    nx.draw_networkx_labels(G,
                            positions,
                            font_size = aes['nodes']['font_size'],
                            font_family= aes['nodes']['font_family'],
                            fontweight = aes['nodes']['font_weight'])
    #Node color bar stuff
    n_color = aes['nodes']['n_color']
    print(n_color)
    node_bar=plt.colorbar(y, label='Module value')
    tick_locs = (np.arange(n_color) + 0.5)*(n_color-1)/n_color
    print(tick_locs)
    node_bar.set_ticks(tick_locs)

    # set tick labels (as before)
    node_bar.set_ticklabels(np.arange(n_color))
    node_bar.set_label('Modularity', fontsize =  aes['nodes']['font_size'])

    ## EDGES ##
    x=nx.draw_networkx_edges(G,
                             positions,
                             edge_list=edges,
                             style='solid',
                             width = weights*aes['edges']['width mod'],
                             edge_color = weights,
                             edge_vmin=aes['edges']['min'],
                             edge_vmax=aes['edges']['max'],
                             edge_cmap= aes['edges']['colormap'])
    # Edge color bar stuff
    sm = plt.cm.ScalarMappable(cmap=aes['edges']['colormap'],
                               norm=plt.Normalize(vmin = aes['edges']['min'], vmax=aes['edges']['max']))
    sm._A = []
    edge_bar=plt.colorbar(sm)

    for l in edge_bar.ax.yaxis.get_ticklabels():
        l.set_size(aes['nodes']['font_size'])
    for l in node_bar.ax.yaxis.get_ticklabels():
        l.set_size(aes['nodes']['font_size'])
        l.set_verticalalignment('center')

    edge_bar.set_label('Strength of edge weight', fontsize = aes['nodes']['font_size'])
    # Title things
    plt.title("Module Connectivity Weights %s"%Type, fontsize = aes['nodes']['font_size']*2)
    plt.axis('off')
    basepath= os.path.join(basepath,'images')
    plt.savefig(os.path.join(basepath,"modularity_%s.png"%(Type)), format="PNG")
    # plt.show()

In [49]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def basic_graph(graph, scl, FONT, metric, size, tile, basepath, bmi, mod):
    #threshold
    e,w = zip(*nx.get_edge_attributes(graph, 'weight').items())
    purr=np.percentile(w, tile)
    g=an.threshold2(graph,purr)
    # style
    fig = plt.figure(figsize = size)
    ax1 = plt.subplot(111, aspect = 'equal')
    plt.subplots_adjust(left=0.25, bottom=0, right=0.75, top=1, wspace=0.2, hspace=0)

    positions=nx.circular_layout(g)
    
    # NODES
    nodes, label = zip(*nx.get_node_attributes(g, 'labels').items())
    if metric == 'null':
        size = 10000
        color = np.array(list(graph.nodes))
        color = np.array(color)
        print(color)
        
    else:
        nodes, color = zip(*nx.get_node_attributes(g, metric).items())
        nodes, size = zip(*nx.get_node_attributes(g, metric).items())
    new={}
    for x,y in nx.get_node_attributes(g, 'labels').items():
        new[x]=y['area']
    
    #draws nodes
    if metric == 'null':
        colz=len(list(set(color)))
        nColormap= plt.get_cmap('Set3', colz)
        print(color.max())
        cm=float(color.min())
        cM=float(color.max())
        n_size=size
    else:    
        color = np.array(color)
        colz=stats.zscore(color)
        nColormap=plt.cm.cool 
        cM=colz.max()
        cm=colz.min()
        scale=scl/colz.max()
        n_size=np.square(colz)*scale
    nx.draw_networkx_nodes(g,
                             positions,
                             node_color=colz,
                             node_size=n_size,
                             alpha=0.8,
                             cmap= nColormap,
                             vmin=cm,
                             vmax=cM)

    #Styling for labels
    nx.draw_networkx_labels(g, 
                            positions,
                            new,
                            font_size=FONT,
                            font_family='sans-serif',
                            fontweight = 'bold',
                            horizontalalignment = 'center',
                            verticalalignment = 'top')


    # EDGES 
    edges,weights = zip(*nx.get_edge_attributes(g, 'weight').items())
    #draw edges
    weights=np.array(weights)
    eColormap=plt.cm.gist_rainbow #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html

    nx.draw_networkx_edges(g, 
                             positions,
                             edge_list=edges,
                             style='solid',
                             width = np.square(weights)*200,
                             edge_color = weights,
                             edge_cmap=eColormap,
                             edge_vmin=weights.min(),
                             edge_vmax=weights.max())
    
    #FInishing touches


    plt.axis('off')
    if bmi == 'no':
        plt.title("%s and edge weights of \n %s graph"%(metric, 'Average BMI'), fontsize = FONT)
    if bmi == 'ov':
        plt.title("%s and edge weights of \n %s graph"%(metric, 'High BMI'), fontsize = FONT)
    if bmi == 'ob':
        plt.title("%s and edge weights of \n %s graph"%(metric, 'Very high BMI'), fontsize = FONT)
    

    axins1 = inset_axes(ax1,
                    width="2.5%",  # width = 50% of parent_bbox width
                    height="100%",  # height : 5%
                    loc='right',
                    bbox_to_anchor=(0.15, 0., 1, 1), #(x0, y0, width, height)
                    bbox_transform=ax1.transAxes,
                    borderpad=1)
    
    axins2 = inset_axes(ax1,
                    width="2.5%",  # width = 50% of parent_bbox width
                    height="100%",  # height : 5%
                    loc='right',
                    bbox_to_anchor=(.25, 0, 1, 1),
                    bbox_transform=ax1.transAxes,
                    borderpad=1)

    sm = plt.cm.ScalarMappable(cmap=eColormap, norm=plt.Normalize(vmin = weights.min(), vmax=weights.max()))
    sm._A = []

    edge_bar=fig.colorbar(sm, cax=axins1, orientation="vertical")
    for l in edge_bar.ax.yaxis.get_ticklabels():
        l.set_size(FONT)
    edge_bar.set_label('Strength of edge weight', fontsize = FONT)
    
    sn = plt.cm.ScalarMappable(cmap=nColormap, norm=plt.Normalize(vmin = cm, vmax=cM))
    sn._A = []
    
    rgb = sn.to_rgba(x=colz)
    save={}
    i = 0
    for x,y in nx.get_node_attributes(g, 'labels').items():
        save[x]=[y['IC'], y['area'], np.round(rgb[i]*255)]
        i=i+1
    
    node_bar=fig.colorbar(sn, cax=axins2, label='Module value')
    for l in node_bar.ax.yaxis.get_ticklabels():
        l.set_size(FONT)
    node_bar.set_label('%s'%metric,fontsize = FONT)
    
    
    basepath='/Users/gracer/Google Drive/HCP/HCP_graph/1200/images'
    if bmi == 'no':
        plt.savefig(os.path.join(basepath,"%s_%s_%s.png"%(metric, 'Average BMI', mod)), format="PNG")
    if bmi == 'ov':
        plt.savefig(os.path.join(basepath,"%s_%s_%s.png"%(metric, 'High BMI', mod)), format="PNG")
    if bmi == 'ob':
        plt.savefig(os.path.join(basepath,"%s_%s_%s.png"%(metric, 'Very high BMI', mod)), format="PNG")
    plt.show()
    return(save)


In [87]:
for IC, dat in subgraph_dict["ov"].items():
    for x, y in dict(dat.nodes(data=True)).items():
#         print(x)
        if y['labels']['area'] == 'VentralAttn':
            print(IC)
            print(x)
            print(y)
#         for key, value in y['labels'].items():
#             print(value)
#             if value['area'] == 'VentralAttn':
#                 print(key)
#                 print(value)

1
36
{'clustering': 0.48313090418353577, 'centrality': 0.007705459574707215, 'PC': 0.5097202460458009, 'sub_modules': 2, 'sub_zDegree': -0.18862530673056171, 'labels': {'label': 'L_VentralAttn_1', 'Unnamed: 2': 'L_SFL_ROI', 'area': 'VentralAttn', 'IC': 'IC_36'}}
3
30
{'clustering': 0.56, 'centrality': 0.008312855183021887, 'PC': 0.7187314788660837, 'sub_modules': 0, 'sub_zDegree': 0.0, 'labels': {'label': 'R_VentralAttn_12', 'Unnamed: 2': 'R_TPOJ1_ROI', 'area': 'VentralAttn', 'IC': 'IC_30'}}


In [12]:
summary_dict['no']['modules']['graph'].nodes(data=True)

NodeDataView({0: {'modules': 0, 'clustering': 0.6437710437710438, 'centrality': 0.0064897671177409555, 'PC': 0.7294094283256284, 'area': 'Visual'}, 1: {'modules': 1, 'clustering': 0.7468599033816425, 'centrality': 0.002973716896117888, 'PC': 0.3876176945290867, 'area': 'Default'}, 2: {'modules': 2, 'clustering': 0.6115384615384616, 'centrality': 0.004613266626234078, 'PC': 0.4343172229469626, 'area': 'Visual'}, 3: {'modules': 2, 'clustering': 0.7252252252252253, 'centrality': 0.0021067150470575765, 'PC': 0.12922174237774797, 'area': 'Visual'}, 4: {'modules': 3, 'clustering': 0.6303191489361702, 'centrality': 0.005504181704497122, 'PC': 0.657435431746648, 'area': 'CinguloOperc'}, 5: {'modules': 4, 'clustering': 0.581081081081081, 'centrality': 0.005469547439511589, 'PC': 0.5756999763868214, 'area': 'Default'}, 6: {'modules': 1, 'clustering': 0.6334693877551021, 'centrality': 0.005565460193765314, 'PC': 0.5552382205595492, 'area': 'FrontoParietal'}, 7: {'modules': 2, 'clustering': 0.6693

In [79]:
subgraph_dict["no"][1].nodes(data=True)

NodeDataView({1: {'clustering': 0.7468599033816425, 'centrality': 0.002973716896117888, 'PC': 0.3876176945290867, 'sub_modules': 0, 'sub_zDegree': 0.0, 'labels': {'label': 'R_Default_26', 'Unnamed: 2': 'R_PGi_RO', 'area': 'Default', 'IC': 'IC_1'}}, 6: {'clustering': 0.6334693877551021, 'centrality': 0.005565460193765314, 'PC': 0.5552382205595492, 'sub_modules': 1, 'sub_zDegree': -1.0299177691155303, 'labels': {'label': 'R_FrontoParietal_10', 'Unnamed: 2': 'R_PFm_ROI', 'area': 'FrontoParietal', 'IC': 'IC_6'}}, 8: {'clustering': 0.7323671497584541, 'centrality': 0.003099278857443236, 'PC': 0.45354467396413656, 'sub_modules': 2, 'sub_zDegree': 0.3687581372959709, 'labels': {'label': 'L_Default_7', 'Unnamed: 2': 'L_PGs_ROI', 'area': 'Default', 'IC': 'IC_8'}}, 11: {'clustering': 0.6914893617021277, 'centrality': 0.003908304402685893, 'PC': 0.3617757807439894, 'sub_modules': 2, 'sub_zDegree': 0.3687581372959709, 'labels': {'label': 'L_Default_20', 'Unnamed: 2': 'L_8Av_ROI ', 'area': 'Default

In [ ]:
for group, mods in wb_con_PC.items():
    print(group)
    for mod, blank in mods.items():
        print(mod)  
        wb_con_PC[group][mod]=basic_graph(subgraph_dict[group][mod], 50000, 50, 'PC', (75,50), 80, basepath, '%s'%group, '%i'%mod)

participation coefficient 

In [28]:
wb_con_PC = {
    'no':{2:{}, 5:{}, 0:{}},
    'ov': {2:{}, 6:{}, 0:{}},
    'ob':{4:{}, 6:{}, 0:{}}
}

for group, mods in wb_con_PC.items():
    print(group)
    for mod, blank in mods.items():
        print(mod)  
        wb_con_PC[group][mod]=basic_graph(subgraph_dict[group][mod], 50000, 50, 'PC', (75,50), 80, basepath, '%s'%group, '%i'%mod)

no
2
5
0
ov
2
6
0
ob
4
6
0


clustering coefficient 

In [29]:
wb_con_CC = {
    'no':{2:{},5:{}},
    'ov': {2:{}, 6:{}},
    'ob':{4:{}, 6:{}}
}

for group, mods in wb_con_CC.items():
    print(group)
    for mod, blank in mods.items():
        print(mod)  
        wb_con_CC[group][mod]=basic_graph(subgraph_dict[group][mod], 50000, 50, 'clustering', (75,50), 80, basepath, '%s'%group, '%i'%mod)

no
2
5
ov
2
6
ob
4
6


Betweenness centrality

In [30]:
wb_con_BC = {
    'no':{0:{}},
    'ov': {0:{}},
    'ob':{0:{}}
}

for group, mods in wb_con_BC.items():
    print(group)
    for mod, blank in mods.items():
        print(mod)  
        wb_con_BC[group][mod]=basic_graph(subgraph_dict[group][mod], 50000, 50, 'centrality', (75,50), 80, basepath, '%s'%group, '%i'%mod)

no
0
ov
0
ob
0


In [31]:
wb_con = {'CC':wb_con_CC, 'PC':wb_con_PC, 'BC':wb_con_BC}

## Get the colors for workbench

In [10]:
pth='/Users/gracer/Google Drive/HCP/HCP_graph/1200/brains/color_brain'

In [20]:
mod_colors ={
0 : [225, 0, 0, 225],
1 :[225, 128, 0, 225],
2 :[225, 225, 0, 225],
3 :[128, 225, 0, 225],
4 :[0, 128, 225, 225],
5 :[127, 0, 225, 225],
6 :[225, 0, 225, 225],
7 :[225, 0, 127, 225],
8 :[0, 225, 225, 225],
9 :[0, 225, 225, 225]}

In [25]:
submod_colors ={
0 : [255, 0, 0, 255],
1 :[255, 25, 0, 255],
2 :[255, 100, 0, 255],
3 :[255, 200, 0, 255 ],
4 :[255, 225, 0, 255 ],
5 :[255, 0, 100, 255],
6 :[225, 0, 200, 255 ],
7 :[225, 0, 255, 255 ],
8 :[0, 0, 255, 255],
9 :[0, 200, 255, 255],
10: [0, 255, 255, 255]}

In [16]:
submods={
    'no':[2,5],
    'ov':[2,6],
    'ob':[4,6]
}

In [26]:
for group, listmods in submods.items():
    print(group)
    for num in listmods:
        writepath = os.path.join(pth,'%s_SUBmod%i_colors.txt'%(group, num))
        f = open(writepath, "w")
        for IC, dat in dict(subgraph_dict[group][num].nodes(data=True)).items():
            mod=dat['sub_modules']
            print(mod)
            f.write('LABEL_%i\n'%(IC+1))
            f.write('%i %i %i %i %i\n'%(IC+1, submod_colors[mod][0], submod_colors[mod][1], submod_colors[mod][2], submod_colors[mod][3]))
        f.close()


no
0
1
0
1
1
1
1
2
1
2
3
2
3
1
0
2
3
3
3
3
3
3
4
5
6
4
0
1
2
3
4
5
6
7
8
9
10
ov
0
1
0
1
1
1
1
1
2
3
2
3
0
0
2
3
3
3
3
3
3
4
5
6
7
0
1
2
3
4
5
6
7
8
ob
0
1
2
3
4
5
6
7
0
0
1
2
3
4
5
6
7


In [14]:
writepath = os.path.join(pth,'%s_SUBmod_colors.txt'%('no'))
f = open(writepath, "w")

for IC, dat in dict(subgraph_dict['no'][2].nodes(data=True)).items():
    mod=dat['sub_modules']
    print(mod)
    f.write('LABEL_%i\n'%(IC+1))
    f.write('%i %i %i %i %i\n'%(IC+1, submod_colors[mod][0], submod_colors[mod][1], submod_colors[mod][2], submod_colors[mod][3]))
f.close()


0
1
0
1
1
1
1
2
1
2
3
2
3
1
0
2
3
3
3
3
3
3
4
5
6
4


In [45]:
for group, graph in subgraph_dict.items():
    print(group)
    writepath = os.path.join(pth,'%s_mod_colors.txt'%(group))
    f = open(writepath, "w")
    for IC, dat in dict(graph['modules']['graph'].nodes(data = True)).items():
        mod=dat['modules']
        f.write('LABEL_%i\n'%(IC+1))
        f.write('%i %i %i %i %i\n'%(IC+1, mod_colors[mod][0], mod_colors[mod][1], mod_colors[mod][2], mod_colors[mod][3]))
    f.close()
    
    
    

no
ov
ob


In [46]:
for measure, _con in wb_con.items():
    print(measure)
    for group, mods in _con.items():
        print(group)
        for mod, data in mods.items():
            print(mod)
            writepath = os.path.join(pth,'%s_%s_%i.txt'%(measure, group, mod))
            f = open(writepath, "w")
            for key, value in data.items():
                print(key)
                x = value[2]
                print(x)
                f.write('LABEL_%i\n'%(key+1))
                f.write('%i %i %i %i %i\n'%(key+1, x[0], x[1], x[2], x[3]))
            f.close()

NameError: name 'wb_con' is not defined

In [27]:
import subprocess
import glob
for x in glob.glob(os.path.join(pth,'*_SUBmod*_colors.txt')):
    name=x.split('.')[0]
    cmd = ['wb_command', '-cifti-label-import', '%s'%(os.path.join(pth,'melodic_IC_ftb_copy.dlabel.nii')), '%s'%(os.path.join(pth,'%s'%x)),'%s'%(os.path.join(pth, '%s.dlabel.nii'%name)), '-discard-others']
    process=subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    process.wait()

In [28]:
for x in glob.glob(os.path.join(pth,'*.txt')):
    name=x.split('.')[0]
    cmd = ['wb_command', '-cifti-label-import', '%s'%(os.path.join(pth,'melodic_IC_ftb_copy.dlabel.nii')), '%s'%(os.path.join(pth,'%s'%x)),'%s'%(os.path.join(pth, '%s.dlabel.nii'%name)), '-discard-others']
    process=subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    process.wait()

## Get ICnames

In [ ]:
g = subgraph_dict['ob'][6]
new={}
for x,y in nx.get_node_attributes(g, 'labels').items():
        new[x]=[y['IC'], y['area']]

In [22]:
subedge_dict.keys()

dict_keys(['no', 'ov', 'ob'])

In [25]:
len(list(subedge_dict['no']['graphs'].keys()))

156

In [30]:
subedge_dict['no']['comm_graph']['144125'].edges(data = True)

EdgeDataView([(0, 0, {'weight': 8.203203}), (0, 1, {'weight': -19.191235000000006}), (0, 2, {'weight': 27.261477000000003}), (0, 3, {'weight': -7.5263100000000005}), (0, 4, {'weight': 0.2746799999999999}), (0, 5, {'weight': -5.137447999999999}), (0, 6, {'weight': -0.8716009999999998}), (0, 7, {'weight': -0.553889}), (1, 1, {'weight': 114.82357900000002}), (1, 2, {'weight': -178.82932600000015}), (1, 3, {'weight': 33.684967000000015}), (1, 4, {'weight': -1.502563999999998}), (1, 5, {'weight': 29.122959999999985}), (1, 6, {'weight': 5.330761000000001}), (1, 7, {'weight': 3.841581000000001}), (2, 2, {'weight': 134.76887399999998}), (2, 3, {'weight': -52.50711099999993}), (2, 4, {'weight': -2.5709169999999983}), (2, 5, {'weight': -57.49361900000006}), (2, 6, {'weight': -9.432249}), (2, 7, {'weight': -7.294679999999998}), (3, 3, {'weight': 44.30100999999999}), (3, 4, {'weight': -16.146562999999993}), (3, 5, {'weight': 17.936660999999997}), (3, 6, {'weight': 2.08734}), (3, 7, {'weight': 2.03

In [49]:
summary_dict['no'].keys()

dict_keys(['mean_FC', 'graphs', 'modules', 'corrmat', 'clustering_coeff', 'btn_centrality', 'PC', 'comm_graph'])

In [31]:
tmp_dict = {'no':{}, 'ov': {}, 'ob': {}}
for group, dat in save_dict['NR'].items():
    print(group)
    x=an.cal_edges(dat['comm_graph'], basepath, group)
    tmp_dict[group]=pd.concat(list(x.values()))
sub_comm_edge_df=pd.concat(list(tmp_dict.values()))


11-18-2020_11-53-43 NumExpr defaulting to 4 threads. 


no
200513
379657
125222
217429
157437
979984
256540
281135
117324
140319
285446
729557
130821
145127
529549
188145
645551
146331
137027
309636
168139
198451
152427
173536
704238
186444
371843
105620
475855
180735
154734
361234
129634
174437
835657
245333
178950
210112
194847
185846
148840
135629
378857
553344
120717
138332
138534
169545
135225
561444
622236
238033
206222
204420
117122
178849
109830
228434
173940
208125
102109
381543
571144
499566
465852
386250
180432
350330
227432
448347
555651
459453
366042
156536
128127
101107
185341
173637
101915
214726
162228
198855
122620
837560
144125
849971
144428
112112
131419
580347
142828
200917
749361
172534
199352
107321
316633
176037
558960
616645
654754
160123
349244
361941
186040
157942
113619
190031
432332
101309
185038
727553
158843
479762
149842
138231
397154
336841
114318
120414
175742
153429
715041
210011
567759
132017
802844
122317
250932
214423
285345
130013
111716
175338
146533
173738
211417
208226
180937
114823
176441
173435
148

# real module graphs with accurate edge weights

In [40]:
sub_comm_edge_df

,source,target,weight,group,z_weight
1,0,1,-1.029504,no,0.182965
2,0,2,1.476587,no,0.331816
3,0,3,3.804431,no,0.470079
4,0,4,-2.606209,no,0.089316
5,0,5,1.496303,no,0.332987
...,...,...,...,...,...
37,5,7,0.260668,ob,0.300060
38,5,8,0.644951,ob,0.348569
40,6,7,-0.048968,ob,0.260974
41,6,8,-0.114537,ob,0.252698


In [62]:
no_ = sub_comm_edge_df[((sub_comm_edge_df['group'] == "no"))]
ov_ = sub_comm_edge_df[((sub_comm_edge_df['group'] == "ov"))]
ob_ = sub_comm_edge_df[((sub_comm_edge_df['group'] == "ob"))]

In [56]:
no_g = nx.from_pandas_edgelist(no_, 'source', 'target', edge_attr=['weight', 'z_weight'])

In [60]:
ov_g = nx.from_pandas_edgelist(ov_, 'source', 'target', edge_attr=['weight', 'z_weight'])

In [63]:
ob_g = nx.from_pandas_edgelist(ob_, 'source', 'target', edge_attr=['weight', 'z_weight'])

In [64]:
an.module_fig(no_g, 'Average BMI z-scored', basepath, aes_dict['no'])
an.module_fig(ov_g, 'High BMI z-scored', basepath, aes_dict['ov'])
an.module_fig(ob_g, 'Very High BMI z-scored', basepath, aes_dict['ob'])

8
8
[0.4375 1.3125 2.1875 3.0625 3.9375 4.8125 5.6875 6.5625]
9
9
[0.44444444 1.33333333 2.22222222 3.11111111 4.         4.88888889
 5.77777778 6.66666667 7.55555556]
9
9
[0.44444444 1.33333333 2.22222222 3.11111111 4.         4.88888889
 5.77777778 6.66666667 7.55555556]


In [41]:
test = sub_comm_edge_df[((sub_comm_edge_df['source'] == 2) & (sub_comm_edge_df['target'] == 5)) & ((sub_comm_edge_df['group'] == "no"))]

In [43]:
test['z_weight'].mean()

-0.9066426219926234

In [48]:
test['weight'].mean()

-21.754986698717957

In [47]:
summary_dict['no']['comm_graph'].edges(data = True)

EdgeDataView([(0, 0, {'weight': 6.839178153846154, 'z_edge': -0.6503044985352724}), (0, 1, {'weight': 3.215265679487179, 'z_edge': 0.20274056564229004}), (0, 2, {'weight': 4.525617647435897, 'z_edge': 0.5503912390299686}), (0, 4, {'weight': 1.2273827051282051, 'z_edge': -0.32466647579075253}), (0, 5, {'weight': 1.4031254807692308, 'z_edge': -0.2780400000124421}), (0, 3, {'weight': 0.5767242564102565, 'z_edge': -0.49729326109354666}), (0, 6, {'weight': 0.08255632692307692, 'z_edge': -0.6284014042909447}), (0, 7, {'weight': 0.08044707692307691, 'z_edge': -0.6289610113284271}), (1, 1, {'weight': 83.02367183974364, 'z_edge': -0.6503044985352724}), (1, 4, {'weight': 13.34028218589744, 'z_edge': 2.8890179114345043}), (1, 5, {'weight': 12.0677711602564, 'z_edge': 2.5514068528958678}), (1, 6, {'weight': 2.1015069358974365, 'z_edge': -0.09275178043822663}), (1, 3, {'weight': 14.172688910256399, 'z_edge': 3.109864497187218}), (1, 7, {'weight': 1.8740285384615385, 'z_edge': -0.1531042811282053}),

In [ ]:
# an.module_fig(summary_dict['no']['comm_graph'], 'Average BMI z-scored', basepath, aes_dict['no'])
an.module_fig(summary_dict['ov']['comm_graph'], 'High BMI z-scored', basepath, aes_dict['ov'])
an.module_fig(summary_dict['ob']['comm_graph'], 'Very High BMI z-scored', basepath, aes_dict['ob'])
